<img src="https://i.imgur.com/XSzy00d.png" style="float:right;width:150px">

**Geoinformationssystem und Python**

# Einleitung

## Lernziele

- Sie wissen, was **georeferenzierte Daten** sind und was ein **GIS** ist
- Sie kennen das Datenformat **GeoJSON**
- Sie können mittels `folium` Daten auf einer **Karte visualisieren**
- Sie kennen **`geopandas`** um GeoJSON Dateien weiterzuverarbeiten

# Georeferenzierte Daten

Ein wichtiger Subtyp von Daten sind die **georeferenzierten Daten**. Solche Daten haben einen Bezug zu einem geografischen Ort. Dieser ist meist über ein Koordinatenpaar angegeben. Viele für Menschen wichtige Daten haben einen Bezug zu einem Ort (Temperatur, Haltestellen, Läden, Point of Interests, etc.) und darum spiele georeferenzierte Daten eine wichtige Rolle.

Systeme, die mit georeferenzierten Daten arbeiten, werden als **GIS** (Grafische Informationssysteme) bezeichnet.

# GeoJSON

> GeoJSON ist ein offenes Format, um geografische Daten nach der Simple-Feature-Access-Spezifikation zu repräsentieren. Dafür wird die JavaScript Object Notation verwendet.
> 
> Mehr Informationen zu GeoJSON gibt es auf der [Wikipedia Seite](https://de.wikipedia.org/wiki/GeoJSON).

## Einführung

Um innerhalb eines Jupyter Notebook dieses Format visuell darzustellen, existiert eine  entsprechendes Modul [`jupyterlab-geojson`](https://pypi.org/project/jupyterlab-geojson/). Da nach der Installation ein Neustarten des Jupyterlab Server nötig ist, wurde das Modul bereits vorgängig installiert. 

In [ ]:
from IPython.display import GeoJSON

Die Funktion `GeoJSON` erwartet einen Parameter, sonst kommt es zu einer leeren Ausgabe:

In [ ]:
GeoJSON()

Der Parameter muss ein valides `GeoJSON` Objekt sein. Hier ein Beispiel:

In [ ]:
GeoJSON({
    "coordinates": [
      7.440608559129629, # longitude
      46.942366944944524 # latitude
    ],
    "type": "Point"
})

Hier übergeben wir also ein Dictionary, welches zwei Keys, `coordinates` und `type` enthält. `coordinates` ist wiederum eine Liste welche zwei Floats enthält welche die [**longitude**](https://de.wikipedia.org/wiki/Geographische_Länge) resp. [**latitude**](https://de.wikipedia.org/wiki/Geographische_Breite) des Punktes repräsentieren. 

<div class="gk-exercise">

<img src="https://i.imgur.com/JyhBeDB.png" class="gk-exercise-image">

<span class="gk-exercise-label">Aufgabe</span> 

- Öffne den folgenden [Link](https://geojson.io) in einem neuen Fenster/Tab. 
- Suche nach **deiner** Adresse und füge ein Marker hinzu
- kopiere das resultierende GeoJSON in die folgende Codezelle

<details>
    <img src="https://i.imgur.com/YdpkyZ9.gif">
</details>
    
</div>

In [ ]:
# YOUR CODE HERE

Dieses Dictionary besteht auf der obersten Ebene aus den Keys `type` und `features`. Der Value von `features` ist eine Liste, dadurch können wir mehrere "Features", d.h. Punkte, Linien, Kreise, usw., auf einer Karte darstellen.

<div class="gk-exercise">

<img src="https://i.imgur.com/JyhBeDB.png" class="gk-exercise-image">

<span class="gk-exercise-label">Zusatz Aufgabe</span>

- Gehe zurück zu [geojson.io](geojson.io)
- Füge auf geojson.io weitere Elemente hinzu
- kopiere das resultierende GeoJSON in die folgende Codezelle
    
</div>

In [ ]:
# YOUR CODE HERE

***

## Suchfunktion

Als nächstes soll eine eigene Suchfunktion umgesetzt werden. Dazu werden die folgenden Konzepte verwendet:

- Zugriff auf eine Web API
- Arbeiten mit einem Dictionary 
- Funktionen definieren

Um die Aufgabe ein wenig zu vereinfachen werden wir uns auf die Schweiz beschränken. Das [Bundesamt für Landestopografie](https://www.swisstopo.admin.ch/de/home.html) bietet uns genau eine solche API an: 

> https://api3.geo.admin.ch/services/sdiservices.html#search

Wir werden die [Location Search](https://api3.geo.admin.ch/services/sdiservices.html#id25) API verwenden. Gemäss den Angaben in der Dokumentation müssen wir gewisse Parameter bei jeder Anfrage übergeben. Entsprechend definieren wir ein `API` Variable:

In [ ]:
API = "https://api3.geo.admin.ch/rest/services/api/SearchServer?type=locations&origins=address,parcel&searchText=" 

> Wir schreiben `API` gross um zu zeigen, dass es sich hier um eine [Konstante](https://realpython.com/python-constants/#user-defined-constants) handelt. Also eine Variable welche ihren Wert zur Laufzeit unseres Programms nicht ändert. 

<details>
  Alles was bei einer URL nach einem <code>?</code> geschrieben wird, wird als <a href="https://wiki.selfhtml.org/wiki/URL-Parameter">Get-Parameter</a> bezeichnet. In unserem Fall sind das die Parameter:
  <ul>
    <li><code>type</code> mit dem Wert: <code>locations</code></li>
    <li><code>origins</code> mit dem Wert: <code>address,parcel</code></li>
    <li><code>searchText</code></li>
  </ul>
  die einzelnen Parameter werden mittels <code>,</code> separiert. 
</details>

Um schliesslich unseren Request `url` zu erhalten fügen wir eine entsprechende Adresse hinzu:

In [ ]:
url = API + "Brückenstrasse 73"

> d.h. wir setzten den Wert für den `searchText` Get-Parameter, siehe Details.

und können dann wie gewohnt mittels dem `requests` Modul eine Anfrage starten:

In [ ]:
import requests
response = requests.get(url).json()

gemäss der [Dokumentation](https://api3.geo.admin.ch/services/sdiservices.html#id26) erhalten wir ein JSON Objekt zurück. Deshalb können wir direkt mit der `json` Methode die Antwort zu einem JSON konvertieren

In [ ]:
response

Das hat genau die Struktur welche in der Dokumentation beschrieben wird. 

<div class="gk-exercise">

<img src="https://i.imgur.com/JyhBeDB.png" class="gk-exercise-image">

<span class="gk-exercise-label">Aufgabe</span> 

Orte/Punkte auf einer Landkarte werden mit zwei Koordinaten angegeben. Im JSON, das wir von unserem Request zurückerhalten, sind diese ziemlich verschachtelt angegeben. Diese Koordinaten (`lon` und `lat`) sollen aus dem JSON extrahiert werden, um anschliessend an unsere Funktion `GeoJSON` übergeben werden zu können.

Schreibe den Code um auf die entsprechenden Werte zuzugreifen:

<details>
    <summary>Tipps</summary>
    Versuche Schrittweise vorzugehen. <code>lon</code> und <code>lat</code> sind Schlüssel des Dictionary <code>attrs</code>, was wiederum ...
    <details>
    ... Element jedes Listenelements des <code>results</code> Schlüssel ist.
    </details>
</details>

</div>

In [ ]:
# YOUR CODE HERE

GeoJSON({
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          lon,
          lat
        ],
        "type": "Point"
      }
    }
  ]
})

Nun können wir das zu einer Funktion zusammenfassen:

In [ ]:
def location_search(search_text):
    # replace ist notwenig, untersuche wie sich die Resultate verändern
    # wenn auf replace verzichtet wird. 
    url = API + search_text.replace(" ", ",")
    response = requests.get(url).json()
    # YOUR CODE HERE
    return lat, lon

***

<div class="gk-exercise">

<img src="https://i.imgur.com/JyhBeDB.png" class="gk-exercise-image">

<span class="gk-exercise-label">Aufgabe</span> 


Verwende die `input` Funktion oder das [Text Widget](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Text) um eine Adresssuche zu implementieren. 
    
</div>

In [ ]:
# YOUR CODE HERE

***

## Zusatz

Hier einige Ideen wie die `location_search` erweitert werden könnte:

- Passe die Funktion an, dass sie `lat` und `lon` für sämtliche Resultate und nicht nur das erste zurückgibt. 
    - Stelle alle **Resultate** auf der Karte dar. 
- Gib eine Fehlermeldung zurück falls kein Standort für eine Adresse gefunden wird

> Solltest du diese Änderungen implementieren, wähle einen anderen Namen für deine Funktion, z.B. `locations_search`. Andernfalls kann es in den folgenden Zellen zu Problemen kommen.

# Folium

> `folium` builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the leaflet.js library. Manipulate your data in Python, then visualize it in on a Leaflet map via folium.
> 
> Weitere Infromationen zu `folium` auf der [Projekt Seite](https://python-visualization.github.io/folium/index.html)

## Einführung

Das Erstellen von GeoJSON Objekten wird zwar durch Tools wie [geojson.io](geojson.io) vereinfacht, richtig praktisch ist dieser Ansatz aber nicht. 

Wir werden nun das Modul `folium` kennenlernen, welches uns Methoden und Funktionen, d.h. eine API, bereitstellt um mit einer Karte zu interagieren. 


In [ ]:
import folium
folium.Map(location=[lat, lon], zoom_start=17.0)

Die Funktion `Map` liefert ein `Map` Objekt zurück. Da es sich um den letzten Ausdruck der Zelle handelt wird diese direkt angezeigt. Wir möchten jedoch mit dem `Map` Objekt weiterarbeiten, deshalb weisen wir das Objekt der Variable `m` zu:

In [ ]:
m = folium.Map(location=[lat, lon], zoom_start=17.0)

dadurch kommt es zu keiner Ausgabe, da der letzte Ausdruck nun eine Zuweisung ist. 

> Das Darstellen der Karte ist ein ressourcenintensiver Prozess. Wir können mit dem Objekt `m` arbeiten und erst zum Schluss dieses anzeigen. 

Um uns ein wenig mit `folium` vertraut zu machen werden wir als erstes unsere `location_search` Funktion verwenden um einen Marker auf der Karte darzustellen:

In [ ]:
search_text = input()
lat, lon = location_search(search_text)
folium.Marker(location=[lat, lon]).add_to(m)
m

Der Ausdruck `m` am Ende der Zelle führt wiederum zur Ausgabe der Karte

<div class="gk-exercise">

<img src="https://i.imgur.com/JyhBeDB.png" class="gk-exercise-image">

<span class="gk-exercise-label">Aufgabe</span> 

- Führe die vorherige Zelle 2-3x aus und suche nach verschiedenen Adressen. 
- Versuche auf der Karte die Marker zu finden
- Was stellst du fest?
    
</div>

YOUR ANSWER HERE

***

Wir können der Funktion `Marker` zusätzlich auch ein `popup` und `tooltip` Parameter übergeben:

In [ ]:
search_text = input()
lat, lon = location_search(search_text)
# Wir erstellen ein neues Karten Objekt
m = folium.Map(location=[lat, lon], zoom_start=17.0)
folium.Marker(location=[lat, lon], popup=search_text, tooltip="Click me").add_to(m)
m

Unter [Quickstart](https://python-visualization.github.io/folium/quickstart.html#Markers) findest du weitere Beispiele von möglichen Markern.

## Daten Visualisieren

Wir können `folium` zusammen mit `pandas` verwenden um Daten auf einer Karte zu visualisieren.

In [ ]:
import folium
import pandas as pd

Neben den beiden Modulen benötigen wir weiter 

- eine GeoJSON Datei
- eine CSV Datei

Für dieses Übung haben wir bereits entsprechende Dateien vorbereitet. 

Um sicherzustellen, das sich nicht Marker usw. von vorherigen Aufgaben einschleichen, erstellen wir eine neue Karte

In [ ]:
kantone_map = folium.Map(location=[46.8, 8.33], zoom_start=7)
kantone_map

Im nächsten Schritt geben wir den Pfad zu einer GeoJSON Datei an.

In [ ]:
kantone_geo = "data/kantone.geojson"

diese beinhaltet die Geometrie der Schweizer Kantone.

> Wird die [Datei](data/kantone.geojson) im JupyterLab geöffnet, können die Daten direkt mittels des GeoJSON Widget betrachtet werden. 

Nun wird eine CSV Datei geladen:

In [ ]:
kantone_csv = "data/kantone.csv"
kantone_data = pd.read_csv(kantone_csv)

Diese können wir mittels pandas analysieren

In [ ]:
kantone_data.head()

Folgendermassen lassen sich die `Density` Informationen auf der Karte darstellen

In [ ]:
folium.Choropleth(
    geo_data=kantone_geo,
    name="choropleth",
    data=kantone_data,
    columns=['CantonNumber', 'Density'],
    key_on='feature.properties.KANTONSNUM',
    legend_name="Density",
).add_to(kantone_map)
kantone_map

Die Ausgabe scheint sehr einfarbig. Bevor wir uns diesem Problem annehmen, betrachten wir was genau passiert. Die Zeile

```python
    columns=['CantonNumber', 'Density'],
```

legt fest, welche Spalten des Dataframes `kantone_data` verwendet werden sollen und mittels

```python
    key_on='feature.properties.KANTONSNUM',
```

wird definiert wie `folium` die Daten des DataFrames, `kantone_data`, mit den GeoJSON Objekten verbinden soll, aus der Dokumentation:

> If data is passed as a Pandas DataFrame, the “columns” and “key-on” gk-keywords must be included, the first to indicate which DataFrame columns to use, the second to indicate the layer in the GeoJSON on which to key the data. The ‘columns’ keyword does not need to be passed for a Pandas series.
>
> [Link](https://python-visualization.github.io/folium/modules.html#module-folium.features)


In diesem Fall soll also die Spalte `'CantonNumber'` mit `'feature.properties.KANTONSNUM'` verbunden werden. 

### Binning

Analysieren wir nun das Farbproblem. Gemäss der Dokumentation:

> Colors are generated from color brewer (http://colorbrewer2.org/) sequential palettes. By default, linear binning is used between the min and the max of the values. Custom binning can be achieved with the `bins` parameter.
>
> [folium](https://python-visualization.github.io/folium/modules.html#module-folium.features) 

Möglicherweise kommt es zu einem Problem weil unsere Werte nicht gleichmässig verteilt sind? Nutzen wir unsere `pandas` Skills für eine Situationsanalyse

<div class="gk-exercise">

<img src="https://i.imgur.com/JyhBeDB.png" class="gk-exercise-image">

<span class="gk-exercise-label">Aufgabe</span> 

Erstelle ein Liniendiagramm der Spalte `Density`

<details>
<summary>Tipps</summary>
    Verwende die <code>.plot()</code> Methode.
</details>

</div>

In [ ]:
# YOUR CODE HERE

***

Tatsächlich befinden sich die meisten Werte im Bereich zwischen `0` bis `1000`. Scheinbar funktioniert das standardmässige `linear binning` in diesen Fällen suboptimal. 

Um nun die farbliche Unterscheidung zu verbessern, können wir mittels der [`quantile`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html) Methode von pandas eine Liste `bins` generieren, welche wir anschliessen der Funktion `Choropleth` übergeben können:

In [ ]:
bins = list(kantone_data["Density"].quantile([0, 0.25, 0.5, 0.75, 1]))

kantone_map = folium.Map(location=[46.8, 8.33], zoom_start=7)
folium.Choropleth(
    geo_data=kantone_geo,
    name="choropleth",
    data=kantone_data,
    columns=['CantonNumber', 'Density'],
    key_on='feature.properties.KANTONSNUM',
    bins=bins,
    legend_name="Density",
).add_to(kantone_map)
kantone_map

Dadurch sind die verschiedenen Werte visuell besser unterscheidbar.

<div class="gk-exercise">

<img src="https://i.imgur.com/JyhBeDB.png" class="gk-exercise-image">

<span class="gk-exercise-label">Aufgabe</span> 

- Wähle eine andere Spalte im Dataframe `kantone_data` und visualisiere diese auf der Karte.
- Evtl. ist ebenfalls ein Binning notwendig, dokumentiere mittels Markdown Zellen deine Überlegungen. 

</div>

In [ ]:
# YOUR CODE HERE

***

# GeoPandas

Als letztes Thema wird GeoPandas vorgestellt:

> GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. [...]
> 
> Weitere Infromationen zu `GeoPandas` auf der [Projekt Seite](https://geopandas.org/en/stable/)

GeoPandas erlaubt uns also unser Wissen von Pandas auf GeoJSON Dateien anzuwenden. Als erstes erstellen wir ein 
GeoDataFrame:

In [ ]:
import geopandas as gpd

df = gpd.read_file('data/kantone.geojson')

Wie gewohnt können wir uns mittels der Methode `head` einen ersten Überblick verschaffen

In [ ]:
df.head()

Unter den letztem Spalten befinden sich die folgenden Spalten: `EINWOHNERZ` und `KANTONSFLA`. Dadurch sollte sich eig. auch die Bevölkerungsdichte pro Kanton berechnen lassen

<div class="gk-exercise">

<img src="https://i.imgur.com/JyhBeDB.png" class="gk-exercise-image">

<span class="gk-exercise-label">Aufgabe</span> 

Erstelle eine neue Spalte `DICHTE`

<details>
<summary>Tipps</summary>
    Die Dichte lässt sich berechnen als: <code>EINWOHNERZ</code> / <code>KANTONSFLA</code>
</details>
</div>

In [ ]:
# YOUR CODE HERE

***

Nun können wir dieses Dataframe für unsere Karte verwenden:

In [ ]:
bins = list(df["DICHTE"].quantile([0, 0.25, 0.5, 0.75, 1]))

kantone_map = folium.Map(location=[46.8, 8.33], zoom_start=7)
folium.Choropleth(
    geo_data=kantone_geo,
    name="choropleth",
    data=df,
    columns=['KANTONSNUM', 'DICHTE'],
    key_on='feature.properties.KANTONSNUM',
    bins=bins,
    legend_name="Density",
).add_to(kantone_map)
kantone_map

Hier kommt es, trotz [Binning](#Binning), für einige Kantone zu einer nicht zufriedenstellenden Ausgabe. 

> In diesem Abschnitt wird erklärt wie in solchen Fällen vorgegangen werden kann. 

Um zu verstehen was hier passiert betrachten wir einen Kanton bei dem die Farbe nicht passt:

In [ ]:
df[df.NAME == "Bern"]

Scheinbar gibt es mehrere Einträge für Bern. Von diesen drei Zeilen enthält jedoch nur eine Werte in den Spalten `EINWOHNERZ` und `KANTONSFLA`:

In [ ]:
df[(df.NAME == "Bern") & df.KANTONSFLA.notnull() & df.EINWOHNERZ.notnull()]

> Mittels `&` lassen sich mehrere Filter auf ein DataFrame anwenden. Alternativ können auch die einzelnen Filter, durch das verwenden von temporären Variablen, angewendet werden, z.B.

```pyhton
df_bern = df[(df.NAME == "Bern")]
df_bern_kantonfla_not_null = df_bern[df_bern.KANTONSFLA.notnull()]
...
```

Das führt zu folgender These:

> Wenn wir nur noch Zeilen betrachten, welche Werte, in den Spalten `EINWOHNERZ` und `KANTONSFLA` enthalten, d.h. `KANTONSFLA.notnull()` und `EINWOHNERZ.notnull()`, erhalten wir die gewünschte Ausgabe auf der Karte:

In [ ]:
tmp_df = df[df.KANTONSFLA.notnull() & df.EINWOHNERZ.notnull()]
bins = list(tmp_df["DICHTE"].quantile([0, 0.25, 0.5, 0.75, 1]))

kantone_map = folium.Map(location=[46.8, 8.33], zoom_start=7)
folium.Choropleth(
    geo_data=kantone_geo,
    name="choropleth",
    data=tmp_df,
    columns=['KANTONSNUM', 'DICHTE'],
    key_on='feature.properties.KANTONSNUM',
    bins=bins,
    legend_name="Density",
).add_to(kantone_map)
kantone_map

Voila!

## Geometrien

Mittels `geopandas` können wir auch auf einzelne Geometrien, d.h. in unserem Fall Kantone zugreifen:

In [ ]:
tmp_df.geometry[1]

> Wir werden ab hier nur noch mit `tmp_df` arbeiten um doppelte Kantone auszuschliessen.

Weiter können wir uns auch den Mittelpunkt einer Geometrie anzeigen lassen

In [ ]:
bern = tmp_df.geometry[1]
bern.centroid

Okei das ist nicht sehr aufschlussreich, aber evtl. hat das Objekt ja interessante Attribute:

In [ ]:
dir(bern.centroid)

Ganz zum Schluss dieser Liste finden wir die Attribute `x` und `y`. Damit können wir etwas anfangen!

# Schlussaufgabe

Füge der vorherigen Karte für jeden Kanton einen Marker hinzu. Dieser soll:

- Sich im Zentrum des Kanton befindet
- Im Popup **mind.** die entsprechende Bevölkerungsdichte anzeigen

<details>
    <summary>Tipps</summary>
    Mittels <code>for kanton in tmp_df.index</code> kannst durch die einzelnen Zeilen iterieren.
    <details>
        <summary>Tipp</summary>
        Um den Mittelpunkt pro Zeile zu erhalten kannst du folgenden Code verwenden: 
        <code>tmp_df.geometry[kanton].centroid</code>
    </details>
    <details>
        <summary>Tipp</summary>
        <code>lon</code> entspricht dem <code>x</code> Attribut und <code>lan</code> dem <code>y</code> Attribut. 
    </details>
    <details>
        <summary>Tipp</summary>
        Um pro Zeile auf die Spalte <code>DICHTE</code> zuzugreifen kannst du folgenden Code verwenden:
        <code>tmp_df['DICHTE'][kanton]</code>
    </details>
</details>

In [ ]:
# YOUR CODE HERE

# Zusammenfassung

In diesem Notebook wurden einige Module aus dem Python Universum vorgestellt, welche sich für das Be/Verarbeiten von geographischen Daten eignen. 

Im ersten Teil wurde das `GeoJSON` Widget vorgestellt und wir bauten eine kleine Adresssuchfunktion. Daraufhin wurde anhand eines konkreten Datensatz aufgezeigt, wie sich mittels `folium` Daten auf Karten darstellen lassen. Zuletzt wurde das Modul `geopandas` verwendet um direkt auf GeoJSON Dateien Berechnungen durchzuführen.

Dabei haben wir nur an der Oberfläche gekratzt. Besonders das Modul `geopandas` bietet noch viele weiter Anwendungsmöglichkeiten, [hier](https://geopandas.org/en/stable/getting_started/introduction.html) ein Einstiegspunkt. 

# Impressum

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg" /></a><br />Dieses Werk ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Namensnennung - Weitergabe unter gleichen Bedingungen 4.0 International Lizenz</a>.

Autoren: [Noe Thalheim](mailto:noe.thalheim@bfh.ch), [Benedikt Hitz-Gamper](mailto:benedikt.hitz@unibe.ch)

## Credits

- https://rsandstroem.github.io/tag/folium.html
- Die GeoJSON Daten wurden aus dem Datensatz [swissBOUNDARIES3D](https://www.swisstopo.admin.ch/de/geodata/landscape/boundaries3d.html#technische_details) von SwissTopo erstellt
- https://python-visualization.github.io/folium/quickstart.html
- https://stackoverflow.com/a/49008506


```
I could tell you a joke about UDP.
But I don't know if you'd get it.
```